<a href="https://colab.research.google.com/github/aoc-ptctech/allinone2022/blob/main/jcplessis/Day_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
class Directory:
  name = None
  parent = None
  childs = None
  child_dirs = None

  def __init__(self, name, parent):
    self.name = name
    self.parent = parent
    self.childs = {}
    self.child_dirs = []

  def du(self):
    return sum(self.childs.values()) + sum([child_dir.du() for child_dir in self.child_dirs])

  def ls(self):
    print(self.name)
    for child_dir in self.child_dirs:
      print(f"  - {child_dir.name}")
    for child_file, size in self.childs.items():
      print(f"  - {child_file} : {size}")

  def filter_dirs(self, rule):
    result = []
    if(rule(self)):
      result.append(self)
    for child_dir in self.child_dirs:
      result.extend(child_dir.filter_dirs(rule))
    return result



class FileSystem:

  current_directory = None
  root = None

  def __init__(self):
    self.root = Directory("/", None)
    self.current_directory = self.root

  def cd(self, destination):
    if destination == "/":
      self.current_directory = self.root
    elif destination == "..":
      self.current_directory = self.current_directory.parent
    else:
      for child_dir in self.current_directory.child_dirs:
        if child_dir.name == destination:
          self.current_directory = child_dir
          break
      else:
        child_dir = Directory(destination, self.current_directory)
        self.current_directory.child_dirs.append(child_dir)
        self.current_directory = child_dir

  def add_child(self, name, size):
    self.current_directory.childs[name] = size

  def pwd(self):
    return self.current_directory.name

  def du(self):
    return self.current_directory.du()

  def ls(self):
    return self.current_directory.ls()


  def parse(self, input):
    for line in input.split("\n"):
      if line.startswith("$ cd "):
        self.cd(line[5:])
      if line[0].isdigit():
        size, file_name = line.split(" ")
        self.add_child(file_name, int(size))
    self.cd("/")

  def filter_dirs(self, rule):
    return self.current_directory.filter_dirs(rule)

  def sum_size(self, rule):
    filtered_dirs = self.filter_dirs(rule)
    return sum([filtered_dir.du() for filtered_dir in filtered_dirs])  

  def space_needed(self):
    needed_for_update = 30000000
    total_fs_size = 70000000
    free_space = total_fs_size - self.root.du()
    return needed_for_update - free_space

  def list_freeable_dirs(self):
    return self.filter_dirs(
        lambda dir : dir.du() >= self.space_needed()
    )
  
  def smallest_freeable_dir(self):
    freeable_dirs = self.list_freeable_dirs()
    return sorted(freeable_dirs, key=lambda dir:dir.du())[0]


In [138]:
import unittest

example = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

class TestAcceptance(unittest.TestCase):

    def test_init_filesystem(self):
      self.assertEqual("/", FileSystem().pwd())

    def test_change_directory(self):
      fs = FileSystem()
      fs.cd("a")
      self.assertEqual("a", fs.pwd())
      fs.cd("b")
      self.assertEqual("b", fs.pwd())
      fs.cd("..")
      self.assertEqual("a", fs.pwd())

    def test_size(self):
      fs = FileSystem()
      fs.add_child("z", 654)
      self.assertEqual(654, fs.du())

    def test_size_recursive(self):
      fs = FileSystem()
      fs.add_child("z", 654)
      fs.cd("a")
      fs.add_child("y", 123)
      self.assertEqual(123, fs.du())
      fs.cd("..")
      self.assertEqual(654 + 123, fs.du())

    def test_init_from_puzzle(self):
      fs = FileSystem()
      fs.parse(example)
      fs.cd("/")
      self.assertEqual(48381165, fs.du())
      fs.cd("a")
      self.assertEqual(94853, fs.du())
      fs.cd("e")
      self.assertEqual(584, fs.du())

    def test_filter_dirs(self):
      fs = FileSystem()
      fs.parse(example)
      fs.cd("/")
      dirs = fs.filter_dirs(lambda x : x.du() < 100000)
      self.assertEqual(2, len(dirs))
      self.assertEqual("a", dirs[0].name)
      self.assertEqual("e", dirs[1].name)

    def test_sum_size_with_filter(self):
      fs = FileSystem()
      fs.parse(example)
      fs.cd("/")
      size = fs.sum_size(lambda x : x.du() < 100000)
      self.assertEqual(95437, size)

    def test_space_needed(self):
      fs = FileSystem()
      fs.parse(example)
      fs.cd("/")
      self.assertEqual(8381165, fs.space_needed())

    def test_list_freeable_dirs(self):
      fs = FileSystem()
      fs.parse(example)
      fs.cd("/")
      dirs = fs.list_freeable_dirs()
      self.assertEqual(2, len(dirs))
      self.assertEqual("/", dirs[0].name)
      self.assertEqual("d", dirs[1].name)

    def test_smallest_freeable_dir(self):
      fs = FileSystem()
      fs.parse(example)
      fs.cd("/")
      dir = fs.smallest_freeable_dir()
      self.assertEqual("d", dir.name)
  
unittest.main(argv=[''], verbosity=2, exit=False)

test_change_directory (__main__.TestAcceptance) ... ok
test_filter_dirs (__main__.TestAcceptance) ... ok
test_init_filesystem (__main__.TestAcceptance) ... ok
test_init_from_puzzle (__main__.TestAcceptance) ... ok
test_list_freeable_dirs (__main__.TestAcceptance) ... ok
test_size (__main__.TestAcceptance) ... ok
test_size_recursive (__main__.TestAcceptance) ... ok
test_smallest_freeable_dir (__main__.TestAcceptance) ... ok
test_space_needed (__main__.TestAcceptance) ... ok
test_sum_size_with_filter (__main__.TestAcceptance) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.022s

OK


In [139]:
#@title Define my input
input = """$ cd /
$ ls
dir ddgtnw
dir dtmbp
dir dzbfsf
dir fwrlqs
305959 jjq.hjd
dir qjnnw
$ cd ddgtnw
$ ls
dir gftgshl
dir grct
57336 tbqpqfgd.wvz
267191 vqms
dir wtgzgmvr
$ cd gftgshl
$ ls
dir mtshhn
dir smnslwd
dir znbs
$ cd mtshhn
$ ls
244930 fsclsm
197930 vnnf
$ cd ..
$ cd smnslwd
$ ls
205127 dbtvp.mbr
dir grct
270601 hcjtjptg
146538 lsqvg.zmm
310443 vnnf
84541 vqms
$ cd grct
$ ls
20977 jjq.hjd
$ cd ..
$ cd ..
$ cd znbs
$ ls
192316 pjrpqc.gwh
5233 tnqpmbjf.prg
$ cd ..
$ cd ..
$ cd grct
$ ls
297156 qzlmfj.lhc
104088 vnnf
$ cd ..
$ cd wtgzgmvr
$ ls
dir cfvjph
dir jzdqctm
153202 slcz
$ cd cfvjph
$ ls
201215 tlms
$ cd ..
$ cd jzdqctm
$ ls
dir hnbjcm
112648 jjq.hjd
319899 lhzjrmsd
118481 pclps
226538 pfnmnm
dir vdzsn
148960 wmvd
dir zvh
$ cd hnbjcm
$ ls
147907 ddgtnw.tpg
dir hgh
107668 qjpfhmw.gts
dir qvnbfdq
dir tlms
dir wlvg
$ cd hgh
$ ls
64701 smnslwd.hnd
$ cd ..
$ cd qvnbfdq
$ ls
36206 dzbfsf.qsr
dir dzfpqb
dir rvdlmnqv
13989 tlms.jgl
145493 vqms
$ cd dzfpqb
$ ls
259317 dzbfsf.mwc
36195 rtgrs.hff
$ cd ..
$ cd rvdlmnqv
$ ls
dir smnslwd
$ cd smnslwd
$ ls
141659 gflcq
315323 mwszdwvg
$ cd ..
$ cd ..
$ cd ..
$ cd tlms
$ ls
263149 gwws.zcb
$ cd ..
$ cd wlvg
$ ls
42151 vqms
$ cd ..
$ cd ..
$ cd vdzsn
$ ls
301547 smnslwd.hzc
$ cd ..
$ cd zvh
$ ls
111901 bsnmdtzq.mjp
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd dtmbp
$ ls
dir cscpfcjv
dir hdjs
dir jcrb
dir spmc
dir spwsfpww
$ cd cscpfcjv
$ ls
260848 qzcffqvp
$ cd ..
$ cd hdjs
$ ls
dir tlms
$ cd tlms
$ ls
174135 fwpwgz
$ cd ..
$ cd ..
$ cd jcrb
$ ls
77666 nqtvhszf
198488 smnslwd
40032 vnnf
$ cd ..
$ cd spmc
$ ls
dir hhwzwqzq
319757 hlgvsbg
dir jbrftqj
$ cd hhwzwqzq
$ ls
43088 mrlnrp.nqs
206555 vnnf
$ cd ..
$ cd jbrftqj
$ ls
296407 flb
315927 fnbvwh.lcf
$ cd ..
$ cd ..
$ cd spwsfpww
$ ls
dir ddgtnw
dir hzb
dir smnslwd
74678 smnslwd.crg
130878 tlms
239208 tlms.hjr
$ cd ddgtnw
$ ls
133732 gqddds
dir grct
240907 lslgfrm.fct
64110 rtgrs.hff
dir tlms
275504 zvwjph.svd
$ cd grct
$ ls
18055 fvnm
$ cd ..
$ cd tlms
$ ls
54154 vqms
$ cd ..
$ cd ..
$ cd hzb
$ ls
178568 hmdszw
132701 pzsfzr.zqz
dir smnslwd
$ cd smnslwd
$ ls
286526 dtcrb.tzw
214568 smnslwd
$ cd ..
$ cd ..
$ cd smnslwd
$ ls
118657 ccmmrv.wgs
$ cd ..
$ cd ..
$ cd ..
$ cd dzbfsf
$ ls
30773 grct.lbw
dir jttqnbvn
188768 pmdjrf.nqc
dir ptbps
dir tllwm
176302 vqms
$ cd jttqnbvn
$ ls
9663 jjq.hjd
$ cd ..
$ cd ptbps
$ ls
132576 jzfs.hpq
dir pwclsbw
$ cd pwclsbw
$ ls
dir dzbfsf
$ cd dzbfsf
$ ls
157147 smnslwd.jcg
$ cd ..
$ cd ..
$ cd ..
$ cd tllwm
$ ls
dir pcbgr
dir smnslwd
dir svc
$ cd pcbgr
$ ls
dir grct
65990 gvlc.ptr
dir jtqg
108430 pljrmjb.hld
dir smtbvvpn
dir vqjdpt
130683 vqms
$ cd grct
$ ls
155740 hcstcmpz
156312 rtgrs.hff
317800 vnnf
$ cd ..
$ cd jtqg
$ ls
151962 rtgrs.hff
181794 vwzsf
$ cd ..
$ cd smtbvvpn
$ ls
272000 fgqp.mhf
dir mqgzwzn
dir mspltflh
dir smnslwd
21242 vnnf
dir vzjfv
dir wnbtllq
$ cd mqgzwzn
$ ls
dir rhp
$ cd rhp
$ ls
297744 jjq.hjd
$ cd ..
$ cd ..
$ cd mspltflh
$ ls
32927 rlwp.phz
109360 vlpzz
$ cd ..
$ cd smnslwd
$ ls
172760 vqjgbzd.glg
$ cd ..
$ cd vzjfv
$ ls
dir dcrlnllf
dir ddgtnw
dir pjhnl
dir twfpbs
$ cd dcrlnllf
$ ls
19043 thrvjqmj.smd
$ cd ..
$ cd ddgtnw
$ ls
301193 fdh.pls
$ cd ..
$ cd pjhnl
$ ls
305148 dqfwn.zhg
dir dzbfsf
34622 smnslwd.nzj
$ cd dzbfsf
$ ls
227330 bdmbnm.zpl
232365 cfmhqlhp.qvj
$ cd ..
$ cd ..
$ cd twfpbs
$ ls
dir rfn
dir vdr
$ cd rfn
$ ls
72138 grct
$ cd ..
$ cd vdr
$ ls
19296 vqms
$ cd ..
$ cd ..
$ cd ..
$ cd wnbtllq
$ ls
283471 grct.vmq
285265 sqqj
248581 trgqdwsc.zjc
$ cd ..
$ cd ..
$ cd vqjdpt
$ ls
271132 hprsbjzw.lnr
$ cd ..
$ cd ..
$ cd smnslwd
$ ls
4812 bcg.vwj
dir ddgtnw
30986 pwl.frb
dir tlms
dir vrwwh
dir wwbdc
304408 wwvvhr
$ cd ddgtnw
$ ls
303604 fsclsm
71614 jlpzljjl.vzw
110905 mzlj.qjz
56751 pnhjdbnt
$ cd ..
$ cd tlms
$ ls
36089 bpnhvpdf.spq
dir ddgtnw
dir grct
97469 jjq.hjd
dir jlz
dir nhvvs
dir ptzptl
dir rsqtp
dir sgbnjmwq
dir smnslwd
$ cd ddgtnw
$ ls
dir hwnfv
dir jsdmffq
268127 ndwhj
255789 qzlfsmm
230625 vnnf
$ cd hwnfv
$ ls
224423 fzrj
$ cd ..
$ cd jsdmffq
$ ls
dir dzbfsf
$ cd dzbfsf
$ ls
283246 zrfjlcg.sct
$ cd ..
$ cd ..
$ cd ..
$ cd grct
$ ls
40705 sgpc.wfv
$ cd ..
$ cd jlz
$ ls
dir vrcb
$ cd vrcb
$ ls
223815 gctvnv.rpb
$ cd ..
$ cd ..
$ cd nhvvs
$ ls
10109 rtgrs.hff
$ cd ..
$ cd ptzptl
$ ls
dir chvn
225860 grct
dir hsp
dir nglr
dir qgbbv
64084 swlgd.cjm
dir tlms
$ cd chvn
$ ls
314968 cbp
dir ddgtnw
282139 ddgtnw.ppr
121887 vnnf
dir wvrzs
$ cd ddgtnw
$ ls
19170 dzbfsf.tmf
160727 wqbdcw
$ cd ..
$ cd wvrzs
$ ls
263372 vnnf
$ cd ..
$ cd ..
$ cd hsp
$ ls
dir dzbfsf
dir pcs
dir rjvmwmgm
179166 rtgrs.hff
284238 sgb.gjc
44485 smlqjjbt.pfb
260588 vqms
$ cd dzbfsf
$ ls
dir ntmzsm
$ cd ntmzsm
$ ls
dir drbl
dir frnvqp
258006 qhqss.hnm
$ cd drbl
$ ls
244040 wccppjd.tcg
$ cd ..
$ cd frnvqp
$ ls
196291 pqwqbrdw
$ cd ..
$ cd ..
$ cd ..
$ cd pcs
$ ls
102603 bvmnrf
219969 tlms
$ cd ..
$ cd rjvmwmgm
$ ls
73837 ddgtnw
dir tlms
dir tnpfpcz
247155 vnnf
dir zmvwl
$ cd tlms
$ ls
183532 vqms
$ cd ..
$ cd tnpfpcz
$ ls
dir jjn
274527 rtgrs.hff
97897 ztpd
$ cd jjn
$ ls
dir nlmt
$ cd nlmt
$ ls
132838 mzdcb.gtf
$ cd ..
$ cd ..
$ cd ..
$ cd zmvwl
$ ls
184809 zzdl.lqq
$ cd ..
$ cd ..
$ cd ..
$ cd nglr
$ ls
216870 clfrgzv.lsh
dir ddgtnw
22735 mhpgvbh.phg
200235 vqms
84345 wjrzwlp
dir ztbjwv
$ cd ddgtnw
$ ls
dir zfdts
$ cd zfdts
$ ls
13693 grct
$ cd ..
$ cd ..
$ cd ztbjwv
$ ls
dir zzrsvbg
$ cd zzrsvbg
$ ls
dir ddgtnw
$ cd ddgtnw
$ ls
258910 tlms.hrh
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd qgbbv
$ ls
dir crc
dir prszvcp
dir qwrnllw
151124 rtqqp.wfv
291754 vnnf
279433 wgsjgqm.zrm
$ cd crc
$ ls
257354 ffg
258517 jjq.hjd
$ cd ..
$ cd prszvcp
$ ls
279284 fnwgcvw.dbg
201788 grct.ssc
dir mtlr
$ cd mtlr
$ ls
dir vztmrn
$ cd vztmrn
$ ls
84994 dzbfsf
$ cd ..
$ cd ..
$ cd ..
$ cd qwrnllw
$ ls
dir nsrhgbt
$ cd nsrhgbt
$ ls
dir smnslwd
dir vsnmq
$ cd smnslwd
$ ls
128046 rtgrs.hff
$ cd ..
$ cd vsnmq
$ ls
15634 zpqp
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd tlms
$ ls
240233 rtgrs.hff
$ cd ..
$ cd ..
$ cd rsqtp
$ ls
dir grct
dir hcsjjss
$ cd grct
$ ls
dir ddgtnw
27078 rtgrs.hff
$ cd ddgtnw
$ ls
235401 hzpr
107837 smnslwd.pqr
$ cd ..
$ cd ..
$ cd hcsjjss
$ ls
dir jhhw
18002 nptmmjz.pgj
dir tlms
$ cd jhhw
$ ls
282291 mcmndtb.mjj
$ cd ..
$ cd tlms
$ ls
159577 tlms.ghp
$ cd ..
$ cd ..
$ cd ..
$ cd sgbnjmwq
$ ls
dir ddgtnw
dir qfc
34331 tlms.mdr
$ cd ddgtnw
$ ls
168852 vnnf
63729 vqms
$ cd ..
$ cd qfc
$ ls
321148 ftjjdg
185489 mlp.ssf
195188 tlms
$ cd ..
$ cd ..
$ cd smnslwd
$ ls
dir dzbfsf
102016 grct.vmc
dir gtjgtd
41304 pbdh
94958 tlms.tcf
178471 vqms
$ cd dzbfsf
$ ls
dir dvvcg
dir dznc
51706 hpzg.rwm
179994 vqms
$ cd dvvcg
$ ls
dir bsswp
$ cd bsswp
$ ls
189827 gpzmg
$ cd ..
$ cd ..
$ cd dznc
$ ls
3424 smnslwd
$ cd ..
$ cd ..
$ cd gtjgtd
$ ls
152369 ddgtnw
$ cd ..
$ cd ..
$ cd ..
$ cd vrwwh
$ ls
308054 grct.hgj
$ cd ..
$ cd wwbdc
$ ls
dir zwvtdc
$ cd zwvtdc
$ ls
dir jqtv
$ cd jqtv
$ ls
dir qdd
165188 smnslwd.hwz
$ cd qdd
$ ls
319798 jjq.hjd
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd svc
$ ls
dir ddgtnw
133886 fsclsm
107226 jjq.hjd
259031 wtnbwg.sct
$ cd ddgtnw
$ ls
124212 tlms.qws
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd fwrlqs
$ ls
45167 fsclsm
42603 qfq.pqh
$ cd ..
$ cd qjnnw
$ ls
228764 gnlhtvzt
dir grct
134749 jbs.jnd
66187 jjq.hjd
dir mqbbh
dir nlwjn
dir rdn
dir shffbpjj
154029 spjc
$ cd grct
$ ls
dir ddgtnw
98292 vnnf
$ cd ddgtnw
$ ls
dir zrb
$ cd zrb
$ ls
244157 vnnf
$ cd ..
$ cd ..
$ cd ..
$ cd mqbbh
$ ls
dir dqmhnq
dir dzbfsf
243785 fgzd.rlv
dir grct
dir gwlcnf
dir rnl
dir slnqt
dir smnslwd
dir tlms
183073 tlms.cvt
dir wgctf
$ cd dqmhnq
$ ls
149913 dzpn.qsg
dir hmfvzjz
198969 rtgrs.hff
$ cd hmfvzjz
$ ls
dir dzbfsf
dir hthgs
$ cd dzbfsf
$ ls
318705 cmgjqnb.wbq
$ cd ..
$ cd hthgs
$ ls
dir ljmsqbvz
$ cd ljmsqbvz
$ ls
304509 dwrcjrw
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd dzbfsf
$ ls
116769 ccgfzf.pmh
$ cd ..
$ cd grct
$ ls
246522 tlms.djc
$ cd ..
$ cd gwlcnf
$ ls
109428 vqms
$ cd ..
$ cd rnl
$ ls
80705 grct.ptv
dir mtwrqwwl
174651 smnslwd.glg
30849 swgp
dir tlms
$ cd mtwrqwwl
$ ls
dir wtmgr
$ cd wtmgr
$ ls
134164 tlms.qmv
316076 vqms
138038 wnv
dir zsmmhfq
$ cd zsmmhfq
$ ls
316061 tlms
$ cd ..
$ cd ..
$ cd ..
$ cd tlms
$ ls
224252 fsclsm
4673 png.ntp
$ cd ..
$ cd ..
$ cd slnqt
$ ls
dir lzzwbcnl
dir mpdrjwgl
$ cd lzzwbcnl
$ ls
dir lhltmghz
$ cd lhltmghz
$ ls
dir hzhfgsfd
212254 lcj.rsh
$ cd hzhfgsfd
$ ls
36945 fsclsm
$ cd ..
$ cd ..
$ cd ..
$ cd mpdrjwgl
$ ls
176456 ctftvdhl.hzz
79819 dzbfsf
248804 jvlglnb.dpw
$ cd ..
$ cd ..
$ cd smnslwd
$ ls
33395 vnnf
$ cd ..
$ cd tlms
$ ls
dir jdvhnpv
317469 lzsrnpd
dir tlms
$ cd jdvhnpv
$ ls
dir pbwh
dir vfn
$ cd pbwh
$ ls
170653 smnslwd
$ cd ..
$ cd vfn
$ ls
dir mcdpp
$ cd mcdpp
$ ls
294474 dzbfsf.hfn
$ cd ..
$ cd ..
$ cd ..
$ cd tlms
$ ls
dir lvqjfj
$ cd lvqjfj
$ ls
250581 rtgrs.hff
$ cd ..
$ cd ..
$ cd ..
$ cd wgctf
$ ls
263836 crthpg.vlr
dir ddgtnw
256310 fsclsm
dir mmlw
dir qzq
dir tlms
209457 vqms
$ cd ddgtnw
$ ls
dir nrfgg
$ cd nrfgg
$ ls
278109 dzbfsf
301844 vqlp.wzt
$ cd ..
$ cd ..
$ cd mmlw
$ ls
dir gsvfvgn
$ cd gsvfvgn
$ ls
4966 ddgtnw.rhb
85130 pld.qtc
$ cd ..
$ cd ..
$ cd qzq
$ ls
89149 pgrgt.jmm
$ cd ..
$ cd tlms
$ ls
84537 bzfznn.cdg
$ cd ..
$ cd ..
$ cd ..
$ cd nlwjn
$ ls
dir ddgtnw
dir psdb
dir vdmlzzgd
dir zdzrn
$ cd ddgtnw
$ ls
6932 dzbfsf.jjg
$ cd ..
$ cd psdb
$ ls
196117 bhwb.mfn
127600 vschrflh.fgp
$ cd ..
$ cd vdmlzzgd
$ ls
5427 llch
$ cd ..
$ cd zdzrn
$ ls
dir lfpltz
$ cd lfpltz
$ ls
157546 sffcz
$ cd ..
$ cd ..
$ cd ..
$ cd rdn
$ ls
133184 fsclsm
186144 mrntdh.spz
117372 spdlb.vmp
245469 vjrjwfwl.zzt
$ cd ..
$ cd shffbpjj
$ ls
dir ccwnzd
dir pcqbmmzt
$ cd ccwnzd
$ ls
143939 zlwqpwdl.hbh
$ cd ..
$ cd pcqbmmzt
$ ls
196568 bqm
dir ddgtnw
169897 jppm.lfw
188545 psm.lml
dir rcnp
dir sjhrvszs
83840 vnnf
dir vvh
dir wzcqnz
dir zfctl
$ cd ddgtnw
$ ls
320675 dzbfsf
$ cd ..
$ cd rcnp
$ ls
60088 fsclsm
dir grct
dir qndvg
153481 rzdzmm.prg
$ cd grct
$ ls
23004 rfgbpbt.mhp
166737 vqms
$ cd ..
$ cd qndvg
$ ls
181934 smnslwd.tpb
$ cd ..
$ cd ..
$ cd sjhrvszs
$ ls
dir gfpzrd
283172 rtgrs.hff
dir tlms
$ cd gfpzrd
$ ls
110715 rltrjpg.lch
117420 vnnf
$ cd ..
$ cd tlms
$ ls
211543 cvcq.lqw
132575 tlms
$ cd ..
$ cd ..
$ cd vvh
$ ls
dir bscbv
dir djzcnld
276126 grct.tjl
dir nlsstb
$ cd bscbv
$ ls
116170 smnslwd.pvj
295190 vqms
$ cd ..
$ cd djzcnld
$ ls
6635 dtnqpfqw.nlj
27153 fzpnfnp.jbt
164286 rlchrtlw
231430 rtgrs.hff
33326 wbfqtpjn.vsq
$ cd ..
$ cd nlsstb
$ ls
91522 dbpdbtvw
221628 nfdgjsp.npf
$ cd ..
$ cd ..
$ cd wzcqnz
$ ls
244122 fcqwl.nwt
dir grct
19950 jjq.hjd
296817 nwcvl
dir smnslwd
$ cd grct
$ ls
dir ddgtnw
dir qglfnbds
dir wrthr
$ cd ddgtnw
$ ls
287254 drwqt
90776 gddwrgh.qls
$ cd ..
$ cd qglfnbds
$ ls
dir qzh
$ cd qzh
$ ls
164079 bswc
207352 dzbfsf
203683 ftdjsfbg.lbl
60925 sgmtn.llc
dir wws
$ cd wws
$ ls
111099 mjq.fjz
$ cd ..
$ cd ..
$ cd ..
$ cd wrthr
$ ls
dir ddgtnw
dir jfv
dir ndwlld
dir nsnz
dir vqql
$ cd ddgtnw
$ ls
180911 dzbfsf
301448 jjq.hjd
dir mdmc
dir nlls
207270 vqms
dir wsctbr
$ cd mdmc
$ ls
267495 dzbfsf.mcl
$ cd ..
$ cd nlls
$ ls
221776 jjq.hjd
$ cd ..
$ cd wsctbr
$ ls
93502 fsclsm
$ cd ..
$ cd ..
$ cd jfv
$ ls
129583 ddgtnw
$ cd ..
$ cd ndwlld
$ ls
217056 gtqc.zvq
$ cd ..
$ cd nsnz
$ ls
dir jhntl
$ cd jhntl
$ ls
103365 trdnzfz
$ cd ..
$ cd ..
$ cd vqql
$ ls
201664 smnslwd.gzj
$ cd ..
$ cd ..
$ cd ..
$ cd smnslwd
$ ls
318911 fsclsm
dir jcvmgzc
101015 rtgrs.hff
$ cd jcvmgzc
$ ls
37799 jjq.hjd
$ cd ..
$ cd ..
$ cd ..
$ cd zfctl
$ ls
dir fmfgjw
dir lbz
$ cd fmfgjw
$ ls
159825 ddgtnw.bhf
210407 jjq.hjd
$ cd ..
$ cd lbz
$ ls
206810 gbcqz.lgw
20178 rtgrs.hff"""


In [140]:
fs = FileSystem()
fs.parse(input)
size = fs.sum_size(lambda x : x.du() < 100000)

to_delete = fs.smallest_freeable_dir()
      
print(f"Sum of all sizes of all directories smaller than 100'000 : {size}")
print(f"We should delete {to_delete.name} in order to free {to_delete.du()} space")


Sum of all sizes of all directories smaller than 100'000 : 1334506
We should delete ptzptl in order to free 7421137 space
